In [ ]:
#| default_exp markdown

# Markdown

> For rich markdown and text rendering 

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
from fhbasecoat.interactive import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from itertools import product
from enum import Enum, auto


In [ ]:
app = FastHTML(exts="ws", session_cookie="mysession")
rt = app.route

In [ ]:
srv = JupyUvi(app, port=8003)

# Components

### Copyable Code

In [ ]:
#| export
def CodeHighlight(src, cls="m-0 p-0 pt-2 leading-none"):
    return Pre(
        Code(src, cls="language-python-custom text-sm", style="background: transparent; overflow: visible; padding: 0;"),
        cls=cls,
    )

In [ ]:
#| export
def CopyableCode(code, lang="python"):
    return Div(
        Div(
            Span(lang, cls="text-xs text-muted-foreground px-3 pt-1 pb-1 mb-0 border border-b-0 rounded-t-lg bg-background"),
            cls="flex pl-2 pt-1",
        ),
        Div(
            CodeHighlight(code.strip(), cls="m-3"),
            Button(
                Icon("copy", cls="size-4 copy-icon"),
                Icon("check", cls="size-4 check-icon hidden text-green-500"),
                cls=f"{ButtonT.ghost} absolute top-2 right-2 size-8 p-0",
                onclick="""
                    navigator.clipboard.writeText(this.closest('.relative').querySelector('code').textContent);
                    this.querySelector('.copy-icon').classList.add('hidden');
                    this.querySelector('.check-icon').classList.remove('hidden');
                    setTimeout(() => {
                        this.querySelector('.copy-icon').classList.remove('hidden');
                        this.querySelector('.check-icon').classList.add('hidden');
                    }, 1500);
                """
            ),
            cls="relative border rounded-lg"
        ),
        cls="rounded-lg bg-muted/50 my-2"
    )

In [ ]:
#| demo
def DemoCopyableCode():
    return Div(
        CopyableCode("def test_func(x:int):\n\treturn x+1"),
        cls="w-96"
    )

In [ ]:
#| preview
pw(
    DemoCopyableCode(),
)

### Text Dot Highlight

In [ ]:
#| export
def TextDotHighlight(text, cls="mx-0.5", before_color="bg-blue-500", after_color="bg-blue-500/40"):
    return Span(
        Span(
            text,
            Icon("info", cls="size-3 inline ml-1 text-muted-foreground"),
            cls="relative z-10"
        ),
        cls=f"relative after:content-[''] after:absolute after:w-[4px] after:h-[4px] after:left-1/2 after:-top-1 after:-translate-x-1/2 after:{before_color} after:rounded-lg after:transition-all hover:cursor-pointer hover:after:{after_color} hover:after:w-[calc(100%+10px)] hover:after:h-[calc(100%+10px)] {cls}"
    )

In [ ]:
#| demo
def DemoTextDotHighlight():
    return Div(
        Span("This is an example of a", TextDotHighlight("TextDotHighlight", cls="mx-1")),
        cls="w-96 h-24 p-4"
    )

In [ ]:
#| preview
pw(
    DemoTextDotHighlight()
)

### Text Popover

In [ ]:
#| export
def TextPopover(content, text, id, side="right", align="center"):
    return Popover(content, id=id, trigger_btn=PopoverTriggerButton(TextDotHighlight(text), pid=id, cls="text"), side=side, align=align)

In [ ]:
#| demo
def DemoTextPopover():
    return Div(
        Span("This is an example of a", TextPopover("Popover contents", "TextPopover", "demo-text-pop", side="bottom")),
        cls="w-96 h-24 p-4"
    )

In [ ]:
#| preview
pw(
    DemoTextPopover()
)

### Render MD

In [ ]:
#| export
def md_header(md_item):
    level = len(md_item) - len(md_item.lstrip("#"))
    sizes = {1: "text-3xl", 2: "text-2xl", 3: "text-xl", 4: "text-lg", 5: "text-base", 6: "text-sm"}
    H = [H1, H2, H3, H4, H5, H6][level - 1]
    return H(md_item.lstrip("# ").strip(), cls=f"{sizes.get(level, 'text-base')} font-bold my-4")

In [ ]:
#| export
def extract_codeblock(codeblock):
    return re.findall(r"```(\w+)(.*)```", codeblock, flags=re.DOTALL)[0]

In [ ]:
extract_codeblock("```python\ntest_code()```")

('python', '\ntest_code()')

In [ ]:
#| export
def md_codeblock(md_item):
    lang, code = extract_codeblock(md_item)
    return CopyableCode(code, lang)

In [ ]:
#| export
def render_span(text, replace_map):
    if replace_map is None: replace_map = {}
    pattern = rf"\b({'|'.join(replace_map.keys())})\b"
    parts = [replace_map.get(p, p) for p in re.split(pattern, text) if p]
    return Span(*parts)

In [ ]:
#| export
def render_md_item(md_item: str, replace_map=None):
    if md_item.startswith("#"):   return md_header(md_item)
    if md_item.startswith("```"): return md_codeblock(md_item)
    return render_span(md_item, replace_map)

In [ ]:
#| demo
def DemoRenderMD():
    md_items = L([
        "# A sample md", 
        "```python\ndef test_sample():\n\treturn 'yes'```", 
        "A replace sample test for samples."
    ])
    replace_map = {"sample": Strong("SAMPLE")}
    return Div(
        *md_items.map(render_md_item, replace_map=replace_map),
        cls="w-96"
    )

In [ ]:
#| preview
pw(
    DemoRenderMD()
)